In [3]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [4]:
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.language import Language
from typing import List, Dict, Tuple, Optional

from  src.pipeline.create_pipline_init import create_nlp_pipeline
from  src.create_dataset import instances_from_amazon
from collections import namedtuple
import json
import gzip

GAZETTEERS_PATH = "/data2/zhanghc/RE/low-resource/src/data/gazetteers/"
nlp = create_nlp_pipeline(gazetteers_path=GAZETTEERS_PATH)

In [5]:
instances = instances_from_amazon("/data2/zhanghc/data_external/amazon_reviews/meta_Electronics.json.gz")

In [4]:
from allennlp.data.tokenizers.word_splitter import JustSpacesWordSplitter
from allennlp.data.tokenizers import Tokenizer, WordTokenizer
import re
from tqdm import tqdm
_tokenizer =  WordTokenizer(word_splitter=JustSpacesWordSplitter())

sentences_list = []

for i in tqdm(range(len(instances))):
    text = instances[i][1]
    doc = nlp(text)
    sentences = list(doc.sents)
    
    for sent in sentences:
        sent_str = sent.text.strip()
        tokenized_text = _tokenizer.tokenize(sent_str)
        text_new  = " ".join([w.text for w in tokenized_text])
        
        doc_sent = nlp(text_new)
        #displacy.render(doc_sent, style="ent", jupyter=True)
        
        ents=[]
        for ent in doc_sent.ents:
            ents.append({"text":ent.text,"start_char":ent.start_char,"end_char":ent.end_char,"label":ent.label_})
        
        speed_unit = re.search('\s\d+\.\d+GHz\s', text_new)
        if speed_unit: 
            ents.append({"text":speed_unit.group(),"start_char":speed_unit.span()[0],"end_char":speed_unit.span()[1],"label":"UNIT_SPEED"})
        if len(ents)> 0:
            sentences_list.append({"docid":instances[i][0],"sent":text_new,"ents":ents})
 

100%|██████████| 296782/296782 [3:10:12<00:00, 26.00it/s]   


In [6]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 

df = pd.DataFrame(sentences_list)
df.to_csv("/data2/zhanghc/RE/low-resource/src/data/entity_annotation.csv")

In [ ]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 

file = r'/data2/zhanghc/RE/low-resource/src/data/entity_annotation.csv'
df_r = pd.read_csv(file)

In [ ]:
pd.set_option("display.max_colwidth", 0)

df_r.head()

In [5]:
texta="Operating System include a 1.6 GHz Cortex A9 Dual Core processor, 1GB DDR3 of system memory and 8GB on-board storage memory and 1024 x 600 resolution."
doc_sent = nlp(texta)
displacy.render(doc_sent, style="ent", jupyter=True)